## 파이썬 

- java
- conda install jpype1
- conda install -c conda-forge jpype1
- pip install konlpy

# MS API를 이용해 FAQ 챗봇을 만들어 보자!!
### 필요한 것들
- 1) 챗봇 기본지식 : 엑셀파일로 질문 대답시트를 작성하여 Q&A메이커에 업로드
- 2) 파이썬 기본 지식 : 자연어 처리를 조금 해야함
- 3) 챗봇 프레임웍 : MS Q&A메이커를 사용해서 질문에 대한 대답을 분류

### 1) 챗봇 기본지식 만들기 
- 자기소개 FAQ챗봇 만들기
- 엑셀 파일로 자신의 나이, 직업, 취미 등에 대한 문답을 작성하기 

In [1]:
import pandas as pd

In [2]:
q_a = pd.read_csv("knowledge_2.csv")
q_a.head()

,분류,질문,대답
0,1,이름이 뭐에요?,니노 막시무스 카이저소제 소냐도르 앤 스파르타 김성근
1,1,성함이 어떻게 되시죠,니노 막시무스 카이저소제 소냐도르 앤 스파르타 김성근
2,1,이름이 궁금합니다.,니노 막시무스 카이저소제 소냐도르 앤 스파르타 김성근
3,1,이름 알려주세요,니노 막시무스 카이저소제 소냐도르 앤 스파르타 김성근
4,1,이름이 뭐니?,니노 막시무스 카이저소제 소냐도르 앤 스파르타 김성근


### 2) MS Q&A메이커 로그인
- http://qnamaker.ai 접속하여 로그인
- Create new service -> 서비스명 입력하고 -> 엑셀파일 업로드 -> Create
- 업로드할 파일은 질문과 답변만 있는 파일이어야함
- 업로드 후 퍼블리시 하면 다음과 같은 api정보를 얻을수 있음

#### API로 질문을 보내서 답변을 받아오자!
- 아래 URL 객체 사이에 90c16509-508f-4eef-afd9-1d639d3671b3 를 끼워넣고
- Ocp-Apim-Subscription-Key: 901f5c6d20b84eab800341bb0f04b26f

In [3]:
key_1 ='90c16509-508f-4eef-afd9-1d639d3671b3'
key_2 = '901f5c6d20b84eab800341bb0f04b26f'

url_1 = 'https://westus.api.cognitive.microsoft.com/qnamaker/v2.0/knowledgebases/'
url_2 = '/generateAnswer'
url = url_1 + key_1 + url_2

headers = {'Content-Type':'application/json; charset=utf-8', 
           'Ocp-Apim-Subscription-Key' : key_2}

In [4]:
import requests

def answer(msg):
    ans = requests.post(url, headers=headers,  json={'question': msg})
    return(ans.json()['answers'][0]['answer'])

In [5]:
answer("나이가 어떻게 되시죠")

'나이는 중요하지 않아! 중요한건 마음가짐이지! 만 19세는 넘었다는것만 알려주지...'

### 3) 성능을 높여보자!
- 질문이 들어왔을때 띄어쓰기 해서 QNA MAKER로 질문을 보내준다면?
- 챗봇의 기반지식도 질문부분을 형태소 분석해서 넣어놓고!
- 띄어쓰기 된 질문을 형태소 분석해서 QNA MAKER로 질문을 보내준다면?
#### 더 잘 대답하지 않을까?!!!

#### 3.1) 띄어쓰기 API를 사용해보자!
- 아주 감사하게도 검색해보면 있다!! 감사합니다!!
- https://github.com/haven-jeon/KoWordSpacing

In [6]:
def spacing(sent):
    spacing = requests.put('http://35.201.156.140:8080/spacing', data={'sent':sent}).json()
    return(spacing)

In [7]:
spacing("사람이너무많이와서후끈하군")['sent']

ConnectionError: HTTPConnectionPool(host='35.201.156.140', port=8080): Max retries exceeded with url: /spacing (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000011C6E749A90>: Failed to establish a new connection: [WinError 10060] 연결된 구성원으로부터 응답이 없어 연결하지 못했거나, 호스트로부터 응답이 없어 연결이 끊어졌습니다',))

#### 3.2) 조사제거하여 질문형태를 단순화하자!

In [ ]:
from konlpy.tag import Twitter
twitter = Twitter()

In [ ]:
twitter.pos("사람이 너무 많이 와서 후끈하군")

In [ ]:
from konlpy.tag import Twitter
twitter = Twitter()

def nlp(question):
    string_analyzed = twitter.pos(question, norm=True, stem=True)
    analyzed = []
    for words in string_analyzed:
        if words[1] != 'Josa':
            analyzed.append(words[0])
    message = ' '.join(analyzed)
    return str(message)

In [ ]:
nlp("밥은 먹고 다니냐")

- 띄어쓰기 후 형태소분석하여 조사없는 질문 생성

In [ ]:
def make_question(msg):
    spacing_msg = spacing(msg)['sent']
    nlp_msg = nlp(spacing_msg)
    return(nlp_msg)

In [ ]:
make_question("밥은먹고다니냐")

#### 3.3) 기존 챗봇 기반지식도 형태소 분석해서 넣자
- 질문만 형태소분석하고 답변은 그대로 놔두면 됨

In [ ]:
q_a['질문'][0]

In [ ]:
nlp_question = []

for i in q_a['질문']:
    nlp_question.append(nlp(i))

In [ ]:
q_a['질문_2'] = nlp_question

In [ ]:
q_a.head()

In [ ]:
q_a.to_csv('q_a_know.csv')

### 4) 최종정리
- 그냥 바로 Q&A메이커에 보내면

In [ ]:
qs = ['이름이뭐니', '무슨일해?', '몇살이냐', '어디살아요', '취미가뭐에요']
for i in qs :
    print(answer(i))

- 띄어쓰기 및 형태소 분석해서 Q&A 메이커에 보내면 

In [23]:
def get_ans(question) :
    nlp_question = make_question(question) # 띄어쓰기 및 형태소 분석
    return(answer(nlp_question))           # Q&A메이커 API에 전달

In [24]:
for i in qs :
    print(get_ans(i))

니노 막시무스 카이저소제 소냐도르 앤 스파르타 김성근
피도 눈물도 없는 광기의 매드 사이언티스트?
나이는 중요하지 않아! 중요한건 마음가짐이지! 만 19세는 넘었다는것만 알려주지...
경기도 수원이야...너 혹시 수원에 산다면...너 내 동료가 되라!!
내 취미는 Rock and Roll~!! Love and Peace!! 난 기타를 좋아해!! 나랑 같이 기타를 치지 않을래?


# 사이킷런 라이브러리로 직접 만들어보자!

In [17]:
data = pd.read_csv("knowledge_2.csv")
data.head()

,분류,질문,대답
0,1,이름이 뭐에요?,니노 막시무스 카이저소제 소냐도르 앤 스파르타 김성근
1,1,성함이 어떻게 되시죠,니노 막시무스 카이저소제 소냐도르 앤 스파르타 김성근
2,1,이름이 궁금합니다.,니노 막시무스 카이저소제 소냐도르 앤 스파르타 김성근
3,1,이름 알려주세요,니노 막시무스 카이저소제 소냐도르 앤 스파르타 김성근
4,1,이름이 뭐니?,니노 막시무스 카이저소제 소냐도르 앤 스파르타 김성근


- 질문을 데이터로 분류를 라벨로 사용해서 학습

In [18]:
x_data = data['질문']
y_data = pd.Categorical(data['분류'])

In [19]:
x_data[0:10]

0        이름이 뭐에요?
1     성함이 어떻게 되시죠
2      이름이 궁금합니다.
3        이름 알려주세요
4         이름이 뭐니?
5    나이가 어떻게 되시죠?
6          몇 살이니?
7       나이가 궁금하니다
8        너 몇살 이야?
9       올해 몇살이시죠?
Name: 질문, dtype: object

In [20]:
y_data[0:10]

[1, 1, 1, 1, 1, 2, 2, 2, 2, 2]
Categories (5, int64): [1, 2, 3, 4, 5]

- CountVectorizer 함수로 TDM생성하고 SVM알고리즘으로 분류

In [21]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from konlpy.tag import Twitter

def get_noun(text):
    tokenizer = Twitter()
    nouns = tokenizer.nouns(text)
    return [n for n in nouns]

In [22]:
cv = CountVectorizer(tokenizer=get_noun)
cv.fit_transform(x_data)

<28x27 sparse matrix of type '<class 'numpy.int64'>'
	with 51 stored elements in Compressed Sparse Row format>

In [23]:
text_clf_svm = Pipeline([('vect', CountVectorizer(tokenizer=get_noun)),
                         ('tfidf', TfidfTransformer()),
                         ('clf-svm', SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, n_iter=5, random_state=42))])

In [24]:
text_clf_svm.fit(x_data, y_data)

C:\Users\kimsu\Miniconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ty='l2', power_t=0.5, random_state=42, shuffle=True,
       tol=None, verbose=0, warm_start=False))])

In [25]:
text_clf_svm.predict(['취미는 뭐에요'])

array([4], dtype=int64)

- 5개 분류에 따른 답변 제시를 위한 함수 생성

In [26]:
answer = ["니노 막시무스 카이저소제 소냐도르 앤 스파르타 김성근", 
          "나이는 중요하지 않아! 중요한건 마음가짐이지! 만 19세는 넘었다는것만 알려주지", 
          "피도 눈물도 없는 광기의 매드 사이언티스트", 
          "내 취미는 Rock & Roll~!! Love & Peace!! 난 기타를 좋아해!! 나랑 같이 기타를 치지 않을래?", 
          "경기도 수원이야 너 혹시 수원에 산다면 너 내 동료가 되라!"]

In [21]:
def svm_asn(question):
    idx = int(text_clf_svm.predict([question]))
    return(answer[idx-1])    

In [22]:
print(svm_asn("취미는 뭐에요"))
print(svm_asn("어디 살아요"))
print(svm_asn("연세가 어떻게 되시죠"))
print(svm_asn("직업이 어떻게 되시나요"))

내 취미는 Rock & Roll~!! Love & Peace!! 난 기타를 좋아해!! 나랑 같이 기타를 치지 않을래?
경기도 수원이야 너 혹시 수원에 산다면 너 내 동료가 되라!
나이는 중요하지 않아! 중요한건 마음가짐이지! 만 19세는 넘었다는것만 알려주지
피도 눈물도 없는 광기의 매드 사이언티스트


In [26]:
svm_asn("직업은?")

'피도 눈물도 없는 광기의 매드 사이언티스트'